In [2]:
import numpy as np
from tqdm import tqdm
import os

In [3]:
print(os.getcwd())
test_motifs = ['AATGATGCA', 'CCCXTAC', 'XAX']
test_seq = 'AGAAGAGACCCGTACAATGATGCAA'

/home/jupyter/CS273B


In [54]:
def encode_seq(seq):
    mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    arr = np.zeros((len(seq), 4))
    for i in range(len(seq)):
        b = seq[i]
        if b in mapping.keys():
            arr[i, mapping[b]] = 1
        else:
            arr[i,:] = 1
    return arr
        
def find_motif(seq, motif):
    starts = []
    for i in range(len(seq) - len(motif) + 1):
        prod = np.multiply(seq[i:i+len(motif),], motif)
        if np.sum(prod) == len(motif):
            starts.append(i)
            
    return starts

def motif_score(seq, motif):
    score = 0
    for i in range(len(seq) - len(motif) + 1):
        score += np.sum(np.multiply(seq[i:i+len(motif),], motif))
    #normalize for length of motif
    return score / (len(motif)*(len(seq)-len(motif)))


In [5]:
t = encode_seq(test_seq)
m = encode_seq(test_motifs[2])
print(find_motif(t, m))

[1, 2, 4, 6, 12, 14, 15, 18, 22]


In [6]:
print(test_seq[22:])

CAA


In [7]:
test_input = np.load('testInput.npy', allow_pickle=True)
test_output = np.load('testOutput.npy', allow_pickle=True)

In [8]:
test_input = np.concatenate(test_input, axis=0)
test_output = np.squeeze(np.concatenate(test_output, axis=0))
print(test_input.shape)
print(test_output.shape)

(651946, 1000, 4)
(651946, 4)


In [4]:
motifs = []
with open('Tfdata.txt', 'r') as f:
    for line in f:
        motifs.append(line.split(' ')[1][:-1].upper())
print(motifs[:10])
encoded_motifs = [encode_seq(m) for m in motifs]

['GTTXCCATGGXAAC', 'CCAXXAGXXGGCA', 'GCAXTXCTGGGAXXTGTAGT', 'TCAGXACCXXGGACAG', 'CAGGTG', 'GCAGXTG', 'CAGCTG', 'TAAAAATAAC', 'GGAAGT', 'ACAAGATA']


In [7]:
counts = np.zeros((4, len(motifs)))
for t in range(4):
    diff = []
    for row in test_output:
        diff.append(row[t] - (row[(t+1)%4] + row[(t+2)%4] + row[(t+3)%4])/3)
    rank = sorted(range(len(diff)), key=lambda k: diff[k], reverse=True)
    target_seqs = test_input[rank[:100],:,:]
    for seq in tqdm(target_seqs):
        for i, m in enumerate(encoded_motifs):
            counts[t ,i] += len(find_motif(seq, m))
print(counts)

NameError: name 'test_output' is not defined

In [14]:
for t in range(4):
    count_diff = counts[t] - (counts[(t+1)%4] + counts[(t+2)%4] + counts[(t+3)%4])/3
    #print(count_diff)
    rank = sorted(range(len(count_diff)), key=lambda k: count_diff[k], reverse=True)
    print(rank[:5])
    print([count_diff[i] for i in rank[:10]])
    print([motifs[i] for i in rank[:10]])

[72, 49, 83, 80, 21]
[60.0, 42.0, 34.33333333333333, 18.0, 12.0, 11.666666666666666, 10.333333333333329, 9.333333333333329, 8.333333333333343, 8.0]
['TXTTTXT', 'CAGC', 'ACCTG', 'ACGC', 'CXCGTG', 'TAAACA', 'CCGGA', 'TCCCXXGXG', 'AATC', 'CAAAGXXC']
[49, 80, 13, 27, 69]
[59.33333333333337, 46.0, 31.66666666666667, 25.66666666666667, 18.66666666666667, 9.333333333333336, 7.333333333333329, 4.666666666666668, 4.666666666666668, 3.666666666666668]
['CAGC', 'ACGC', 'CCGGA', 'CACCT', 'TCCCXXGXG', 'CAAAGXXC', 'GGTCA', 'CACGTG', 'CACGTG', 'TGTTCT']
[25, 53, 47, 52, 68]
[54.0, 48.33333333333333, 42.0, 18.66666666666667, 17.0, 15.0, 13.666666666666664, 12.333333333333336, 11.333333333333329, 11.0]
['CCAT', 'TGTXCT', 'AACAA', 'TGACC', 'TTACXXXA', 'AATC', 'CAGCTG', 'CCAAT', 'GGTCA', 'GGAAGT']
[48, 64, 59, 28, 56]
[33.0, 31.0, 28.333333333333336, 28.33333333333333, 26.666666666666657, 24.333333333333336, 20.333333333333332, 18.0, 15.0, 14.333333333333332]
['TGAC', 'AATC', 'TTACXT', 'GATA', 'TCTGG', '

In [12]:
sorted_diff = sorted(diff, reverse=True)
ind = 0
for i in range(len(sorted_diff)):
    if sorted_diff[i] < 50:
        ind = i
        break
print(ind)
target_seqs = test_input[rank[:i],:,:]

60


In [13]:
common_motifs = np.zeros(len(motifs))
for seq in tqdm(target_seqs):
    for i, m in enumerate(encoded_motifs):
        common_motifs[i] += len(find_motif(seq, m))
print(common_motifs)


100%|██████████| 60/60 [01:02<00:00,  1.03s/it]

[  0.   0.   0.   0.  28.  19.  26.   0.  20.   0.  10.   0.   0.   7.
   0.   0.   3.   0.  27.   0.   0.  16.  13.   0.  20. 226.   0.  70.
 193.   0.   0.   0.   1.   0.   3.  10.   0.   1.  29.   0.   2.  13.
   0.   0.   0.  45.   2. 113. 187. 239.  10.  50.  38.  83.   5.   0.
  93.  27.   0.  77.  10.  29.   0.   1. 241.  23.   0.   1.  58.   7.
   3.   0. 380.  24.   1.   5. 123.   0.   0.   2.  24.  38.   0.  67.
   6.  26.  20.  16.  31.   0.  36.  41.   4.   0.  23.   0.   7.   0.
   8.]


In [55]:
s = 0
for i in range(4):
    test_seqs = np.load('cell'+str(i)+'_seqs.npy')[:10] 
    #common_motifs = np.zeros(len(motifs))
    for seq in tqdm(test_seqs):
        for i, m in enumerate(encoded_motifs):
            #common_motifs[i] += len(find_motif(seq, m))
            s += motif_score(seq, m)
    #print(common_motifs)
    #s += np.sum(common_motifs)
print(s/40)


100%|██████████| 10/10 [00:07<00:00,  1.35it/s]

100%|██████████| 10/10 [00:07<00:00,  1.36it/s]

100%|██████████| 10/10 [00:07<00:00,  1.37it/s]

100%|██████████| 10/10 [00:07<00:00,  1.34it/s]

31.150973445860068


In [ ]:
s = 0
for i in range(4):
    test_seqs = np.squeeze(np.load('class_'+str(i+1)+'_seqs.npy'))
    
    t = np.zeros((100, 1000, 4))
    for i in range(len(test_seqs)):
        seq = test_seqs[i]
        max_activations = np.argmax(raw_input_seq, axis=1).flatten()
        encoded_seq = np.zeros(raw_input_seq.shape)
        encoded_seq[np.arange(encoded_seq.shape[0]), max_activations] = 1
        t[i,:,:] = encoded_seq

    #common_motifs = np.zeros(len(motifs))
    for seq in tqdm(t):     
        for i, m in enumerate(encoded_motifs):
            s += motif_score(seq, m)
            #common_motifs[i] += len(find_motif(seq, m))
    #print(common_motifs)
    #s += np.sum(common_motifs)
print(s/400)


 51%|█████     | 51/100 [00:38<00:44,  1.10it/s]

In [61]:
s = 0
for i in range(1):
    test_seqs = np.zeros((10, 1000, 4))
    for i in range(10):
        raw_input_seq = np.eye(4)[np.random.choice(4, 1000)] # define an initial random image
        test_seqs[i,:,:] = raw_input_seq
    
    #common_motifs = np.zeros(len(motifs))
    for seq in tqdm(test_seqs):    
        for i, m in enumerate(encoded_motifs):
            s += motif_score(seq, m)
            #common_motifs[i] += len(find_motif(seq, m))
    #print(common_motifs)
    #s += np.sum(common_motifs)
print(s/10)


100%|██████████| 10/10 [00:07<00:00,  1.35it/s]

31.119905199663464


In [58]:
strs = ['GGACTATAATTAGTACATTATTGAATAGCCAAATATTTAAAAATAGGTGAACTTAATTATCTAGATTGGTAGGCTAAATAAAAAAAAAATAAAATTTATCGATAAATAGCTTGTGTCATTCTAAGATAAATACAGAAATTTATTTTTATTAATTATATAAGATCAACTAAACTCTATTATAAGTTTAAATACTTGGCTAAATTCGGAGATCCACAATGTTTTATTTTGCAATTATTTTTTAACGTCTAATTAAATAGCCAATCACAGAACTTATATCATGAAATTATAGAATGGGGATAGATGATAGCATTAATACAGCTTTAGGCACTAGAAAAGCTCCAAAGAAAATGCCAGGATTTAAGTAATTTAAATATGTAGGTCATATCGCAACTTTATTATTAGGAGTCGGAGAAATGGTATAAACAATCGATTTAGATTATGAGAGGTCGAGAGTAATATATTAAAAGAGAAGAATTAGCCATGTAAATATTTATTAAAAAATTAATATAACAATACTAAGACAGCATAGATAGCAGTAATATAGTACTATAGCTACAGTATATATTATATGTGACCTATCTAAATTTGCGAATGATATCACTTAAACAAGAATGGAGTGTTAAAATTTGAATAACAGCTAGAAAATATATAATAAACAAAGAAGATTATTAAAGATGCAATGAAATTAGCTATATTATAAAATTAAATAAATTGTTGATCTATATTAATAATTATTATCAAAGTAGAATATTAGGACTAGAATGGAAGTTAGATTTGTGAAAATTCATTTATATAATACAGTAGAATAGATAATGTGTGTAATCTAAAAATGAATGAAGATAAATTAAAAGCAACTATAATAGTGACTTAGAGATCTTCTTGATCACTATGATAGCACAATTAGGAATTTTTGCTAGACGATAATGAACTTTAAACTAGTGAATAAATATTTTTACAGATAATATGAATTCTTGCTAAGGAATATTAGGATTGCCACATA',
        'AAAAAAATCGCGCAATACGACTGGACTAGTTGATTTAAAAAACCGAATAAAAAGTAAAGCGTCAGAACACTATTAAAAGAAACTTACCGAAAATAACTAAAAAAATCAAATAACAACGGCACACGGTTTAAAATAAAAACCAACCAGTAATTCGTAACCTTAATCCGGAAATTTAGAATAGTAAAAAAAAAAAAAGGAAACTAACCCAAAAGTACGTTTAACCAGACTAAACGCGGAAATCAAACGAGATGCTATAAAAAAAAGAAAAGTACGTAACTCACCATAACCCGTATAAAACAGTTATTTAAAAACCAAGTAACAAGCGTAAAAAAACCTAAAGCCTTTTTATAAAACATACAGAGATTCGCCTCAAAACGTATACTTAAAAACCGATAAAAAACGCATTTCAGAAAGCTTCCTACATTACCACCGAAACGAAGTAGTAAAAACAGCACTAAAAAAATAAAAAAACAAAAAAGAAGCGCAAACACCCTCTACGTACCAAAAAAACCGCCAAACGATCATTACGGCCAATTCAGATTCAAACAAACAACACTAAAAAAAAATTTTAAACTTACAATGTACCTTACCTAATTACATCTAACTAAGTACTGAACAAAAAAACAAAACACCACACTACACATACCGACCGTCCGTTTGAAAAAAAAAACAATTAAAAACGAAAGAAAACGTTAAACCATTAAACATTAACTGACCAAAAAAACTCCAAAAATTACGACAAAAAACGAAAAAACGTAGTTTATTGTAGACCCCCCCACACCCGCTATCTACGGCGCACAATAGAATTCCTTTTAGAAACTAAAAAGATAAAAAAAAGCATTCAAAAACAGAATGGGGCTTTACAAAATAAGTAAAGTAGCCAAACGATATTAAACAAAACCATTTCGTTAGCAATACCCTAATCTCACTTAATTATAAATACGAACTTTTTACGTTTACCGAAACCCCCAACTGCAGTACTACTTAAAGCGATGCAAAC',
        'TTGAACGTAATACCTCATAGAAGTAATTAATGATTATCTGTTGTGGCCGGGAAGGATAATAGAAAGACTTTTTATATAATAATTTAAAATGTAGCTGTAAAAGTATACTTTGGCCGAAATCAAAAATTTAATTTTTTTGATCTTACATTTCTAAGGTATTTTTTTATTTAATATTTTTTGTTTCTTATTTTTTTTTTACCTTGAAACTCCTGTAAGTTATCTTTTAAATTTTTTATAAGGTTCGTTGTTTTAATTATTTTTGTATGTGTACGAGATTGTATTAAATGTAGTTTATCTTGTTTTCCCTTGGGAGTTTAATCTCTAGCTACCGTCGTTGAATTGCACACTTGTCCCACTATTATTGTCATTGAAACTAATAATAATTCAAGGCTGGTATATGGGAGAGTTGAGCTTTTTGATCTCTAGCTATTCTAATTTTAATTAATATAGTATATAGAGAGCTAGTAAGAGTGCATAATCATGCTATACTAGTGAGTGTGAGTATATTATTAAGTAAGTTCTATTCTAGTATAAATATGGCTTTTTTTGATATCCAATAGATGAGAACTGAGAGAAACGTCTCTGTTATCATGGCTAGTAAGATAAAGTAGCCCAATGACTAGTAATTTCAATTTCAAGTTCGCGGCACATTTCTCCCATTTCGTAAATTAACGTATATTAAAAATTTCTGGCTATTTTATTATATTTATATCTTCCCGACAACACATGGATTTTTTTCCTGATCACCAAACTCTTATTTTTTTTTTTATCTGTGCTATTTAAAAAACTAATTTTTGTGAGTATATTTGATTGAATAGTTCTGATCTATTATGGATCTTTCTACACTTAGTTGTTTTATCGGATTTATTTAAGAGGCAATTAAAAGAGACAGATTTGATGTTCTGACTTTGACTCTTTTTGTTTATTGCATAGTTGACTACTGAATATTAAGTTCGATGGCTTTTTGACGCATTATCTTAGTAGTTCGTTATTGGATTCG',
        'TGTTATGTTTATGCTTCTCCATTATGAATAATAGTGTGTACATTTGTGGTTTAACTTGATTATGTAACACATTGACCCGACATTTCGTTTTAAATGAGATATAAAAGGGATTCATGATTGCTTGTTCTTTCTTGGTTCCGTTAATGGGCATAATCAAGTATCTAGTTCAGTAGGGGGGAATGCCTTGGTGGGAAACGGTAAGGGGAATCGCTACCCATCTGCCATGACCTAGAGACTGTCACTCGATCACCATCAGGATCCGTGAATTATATTCCCAATTCGCCACCTCCGCTCTGCGGCTGAGTACAAGAGTTGCGCGTAGTACCAAACACAGCCACTTTAGTACGAAAATGGAGTGTAAATTTTTTTAGAACCCATCGCCTGGGTAAACAGCTTATGTCTCAGAGAAGCTAGGTTAAATGGTTCCAAAGTTGTCGGCTTTTAATAAGTAAGATGAGGATGTAAGACAAGAGAGAAAAAAGAGACAAACGACGCGAAAGAGGGAAGCATTTTCGTGTCGGACGCGAGATAATATAGGGAGGGTGTGAGTTAATTATCGAGTAAAACGCATACATGTTTTGGTGTCTGAAAAACATGGGGTAACAAGTTATCAGAGCACGCGAGCAAGTGGATGCTCTACCGATAACAATGCTATACAAAAACATAATATGGAAGACATGCAGTAGGGTTGAGGATGAGTAAATTATGGGAGGAGAATGGAGGGAGGGAATGATTGTACGAAAATTGCTATCGACTAAGAAGTCATTCCTTAAGAACCACGGTATTAAAACAGGCATGGGGGATGTAAATTCGCGTAAATAAACCTCAAATCACGTGTACGGAAATACAGTAGGATTTCCTAGAAAGCCTATGAGAAACGAAATTCTGAGTGACTAGAAACATTAAGTGATTACCAGTAATTTTGGTCCCTGGCAAATGTGGAGTAGCCTAACCAGTAGTCTGTGATTATATGCGGGTAGGTGAAAGTAACTTAATAGAT']

In [60]:
s = 0
for i in range(4):
    test_seqs = np.expand_dims(encode_seq(strs[i]), axis=0)
    
    #common_motifs = np.zeros(len(motifs))
    for seq in tqdm(test_seqs):    
        for i, m in enumerate(encoded_motifs):
            s += motif_score(seq, m)
            #common_motifs[i] += len
            #common_motifs[i] += len(find_motif(seq, m))
    #print(common_motifs)
    #s += np.sum(common_motifs)
print(s/4)


100%|██████████| 1/1 [00:00<00:00,  1.50it/s]

100%|██████████| 1/1 [00:00<00:00,  1.51it/s]

100%|██████████| 1/1 [00:00<00:00,  1.38it/s]

100%|██████████| 1/1 [00:00<00:00,  1.51it/s]

32.29498813764546
